In [ ]:
%load_ext autoreload
%autoreload 2
%pdb on
import pdb

In [ ]:
!pip install "stable-baselines3[extra]>=2.0.0a4"

In [ ]:
![[ -d "reinforcement_learning" ]] && pushd reinforcement_learning; git pull; popd
![[ ! -d "reinforcement_learning" ]] && git clone https://github.com/tianhuil/reinforcement_learning.git
!pushd reinforcement_learning; git show HEAD; popd

In [ ]:
import time
from stable_baselines3 import A2C
from stable_baselines3.common.env_checker import check_env
from reinforcement_learning.src.logistics import Logistics
from stable_baselines3.common.monitor import Monitor

env = Logistics()
check_env(env)
log_dir = "/tmp/gym/{}".format(int(time.time()))
env = Monitor(env, log_dir, allow_early_resets=True)


model = A2C("MultiInputPolicy", env, verbose=1).learn(5000)